In [1]:
#This part of code, the goal is to go from mapping and county level weather data to to subgroup data
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path = '/Users/ansonkong/Downloads/rcp85hotter/'
full_historical_data_path = '/Users/ansonkong/Downloads/historic/'

#this just smooth out operation
cutoff_year=2020 #The year prediction dataset start
start_year=2000 #The year historical 
end_year=2100

#Debug mode(only output once)
debug=False

The goal of this version is no need for mapping

In [2]:
#import require file
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime
from glob import glob

ba_dir = data_and_output_dir+'Input/EIA 930/BA/'


# Function to read and merge CSVs from a directory
def read_and_merge_csvs(directory, cols):
    combined_df = pd.DataFrame()
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, file), usecols=cols)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

ba_cols = ['UTC Time at End of Hour', 'Balancing Authority', 'Demand (MW)']
ba_combined_df = read_and_merge_csvs(ba_dir, ba_cols)



#Path, this will update itself
mapping_path = data_and_output_dir + 'output/merged_rb_control_area_mapping.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path = data_and_output_dir + 'input/county_populations_2000_to_2020.csv'
output_path = data_and_output_dir + 'output/'

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)

#crate looping for interp
years = np.arange(cutoff_year, end_year + 1, 10)
interp_years = np.arange(cutoff_year, end_year + 1)

# Ensure FIPS codes are five zero-padded strings
historical_population_df['FIPS'] = historical_population_df['county_FIPS'].astype(str).str.zfill(5)
future_population_df['FIPS'] = future_population_df['FIPS'].astype(str).str.zfill(5)
mapping_df['GEOID'] = mapping_df['GEOID'].astype(str).str.zfill(5)

# Rename 'pop_{year}' columns to just '{year}'
for year in range(start_year, cutoff_year+1):
    if f'pop_{year}' in historical_population_df.columns:
        historical_population_df.rename(columns={f'pop_{year}': str(year)}, inplace=True)



# Assuming 'FIPS' is the column to join on and 'Year' is the column indicating the year in future_population_df
for index, row in future_population_df.iterrows():
    pop_values = [row[str(year)] for year in years if str(year) in row]
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    for year in interp_years:
        future_population_df.at[index, str(year)] = interpolated_values[year - cutoff_year]


# Select relevant columns for historical data
historical_population_df = historical_population_df[['FIPS'] + [str(year) for year in range(start_year, cutoff_year)]]

# Select relevant columns for future data
future_population_df = future_population_df[['FIPS'] + [str(year) for year in interp_years]]

# Concatenate historical and future dataframes
combined_population_df = pd.merge(historical_population_df, future_population_df, on='FIPS', how='outer')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_2361/1708899298.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_2361/1708899298.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0

In [13]:
combined_population_df

,FIPS,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,01001,43872.0,44434.0,45157.0,45762.0,46933.0,47870.0,49105.0,49834.0,50354.0,...,47128.091329,46729.282550,46330.473771,45931.664993,45532.856214,45134.047435,44735.238656,44336.429878,43937.621099,43538.812320
1,01003,141358.0,144988.0,148141.0,151707.0,156573.0,162564.0,168516.0,172815.0,176212.0,...,155272.746623,154106.922068,152941.097514,151775.272959,150609.448404,149443.623850,148277.799295,147111.974740,145946.150186,144780.325631
2,01005,29035.0,29223.0,29289.0,29480.0,29458.0,29452.0,29556.0,29736.0,29836.0,...,25744.305537,25587.176212,25430.046888,25272.917563,25115.788239,24958.658914,24801.529589,24644.400265,24487.270940,24330.141615
3,01007,19936.0,20942.0,20983.0,21037.0,21208.0,21375.0,21285.0,21485.0,21589.0,...,19389.482172,19131.358842,18873.235512,18615.112181,18356.988851,18098.865521,17840.742190,17582.618860,17324.495530,17066.372199
4,01009,51181.0,51999.0,52775.0,53726.0,54469.0,55035.0,55978.0,56866.0,57794.0,...,36979.872051,36458.472746,35937.073440,35415.674134,34894.274829,34372.875523,33851.476217,33330.076912,32808.677606,32287.278300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,56037,37484.0,36566.0,36831.0,36559.0,36854.0,37266.0,37948.0,39320.0,39942.0,...,43773.873286,43518.100045,43262.326803,43006.553562,42750.780321,42495.007079,42239.233838,41983.460597,41727.687355,41471.914114
3141,56039,18367.0,18594.0,18727.0,18890.0,19238.0,19343.0,19666.0,20073.0,20541.0,...,17662.525449,17455.720106,17248.914764,17042.109422,16835.304079,16628.498737,16421.693395,16214.888052,16008.082710,15801.277368
3142,56041,19662.0,19406.0,19567.0,19440.0,19419.0,19431.0,19631.0,20071.0,20537.0,...,19397.176994,19140.941866,18884.706737,18628.471608,18372.236480,18116.001351,17859.766222,17603.531094,17347.295965,17091.060836
3143,56043,8257.0,8023.0,7890.0,7822.0,7754.0,7763.0,7671.0,7805.0,7807.0,...,8006.042822,7950.324806,7894.606790,7838.888773,7783.170757,7727.452740,7671.734724,7616.016708,7560.298691,7504.580675


In [10]:

serives_area_path = data_and_output_dir + '/Input/service_area'
# Read the CSV file
serives_area_df = read_and_merge_csvs(serives_area_path, ['BA_Code','County_FIPS','Year'])
serives_area_df['BA_Code'].unique()

array(['NBSO', 'AEC', 'AMPL', 'AZPS', 'AECI', 'BPAT', 'CISO', 'CPLE',
       'CHPD', 'CEA', 'DOPD', 'DUK', 'EPE', 'ERCO', 'FPL', 'FPC', 'GVL',
       'HST', 'IPCO', 'IID', 'JEA', 'LDWP', 'LGEE', 'NWMT', 'NEVP',
       'ISNE', 'NSB', 'NYIS', 'OVEC', 'PACW', 'PACE', 'FMPP', 'GCPD',
       'PJM', 'PGE', 'PSCO', 'PNM', 'PSEI', 'BANC', 'SRP', 'SCL', 'SCEG',
       'SC', 'SPA', 'SOCO', 'TPWR', 'TAL', 'TEC', 'TVA', 'TIDC', 'HECO',
       'WAUW', 'AVA', 'SEC', 'TEPC', 'WALC', 'WACM', 'SEPA', 'MISO',
       'SWPP', 'EEI'], dtype=object)

In [21]:
ba_combined_df

,Balancing Authority,UTC Time at End of Hour,Demand (MW)
0,AEC,01/01/2018 7:00:00 AM,748
1,AEC,01/01/2018 8:00:00 AM,763
2,AEC,01/01/2018 9:00:00 AM,789
3,AEC,01/01/2018 10:00:00 AM,814
4,AEC,01/01/2018 11:00:00 AM,848
...,...,...,...
3158641,YAD,01/01/2019 1:00:00 AM,NaN
3158642,YAD,01/01/2019 2:00:00 AM,NaN
3158643,YAD,01/01/2019 3:00:00 AM,NaN
3158644,YAD,01/01/2019 4:00:00 AM,NaN


In [20]:
import pandas as pd

# Assuming serives_area_df and combined_population_df are already defined

# Step 1: Add a placeholder for the population lookup
serives_area_df['Population'] = 0
serives_area_df['County_FIPS']=serives_area_df['County_FIPS'].astype(str).str.zfill(5)
# Step 2: Populate 'Population' in serives_area_df by looking up in combined_population_df
for idx, row in serives_area_df.iterrows():
    county_fips = row['County_FIPS']
    year = row['Year']
    
    # Find the matching row in combined_population_df by 'FIPS'
    matching_population_row = combined_population_df[combined_population_df['FIPS'] == county_fips]
    if not matching_population_row.empty and str(year) in combined_population_df.columns:
        # If a matching row is found and the year column exists in combined_population_df,
        # retrieve the population value
        population_value = matching_population_row.iloc[0][str(year)]
        serives_area_df.at[idx, 'Population'] = population_value

# Step 3: Compute 'Total_Population' for each combination of 'Year' and 'BA_Code' in serives_area_df
serives_area_df['Total_Population'] = serives_area_df.groupby(['Year', 'BA_Code'])['Population'].transform('sum')

# The 'Total_Population' column now contains the summed population for each combination of Year and BA_Code
serives_area_df


/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_2361/2799943440.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '74434.99509211467' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  serives_area_df.at[idx, 'Population'] = population_value


,Year,County_FIPS,BA_Code,Population,Total_Population
0,2020,23003,NBSO,74434.995092,3.757132e+05
1,2020,23009,NBSO,53103.777044,3.757132e+05
2,2020,23019,NBSO,158886.075450,3.757132e+05
3,2020,23021,NBSO,18456.013671,3.757132e+05
4,2020,23027,NBSO,38974.016788,3.757132e+05
...,...,...,...,...,...
26339,2015,48483,SWPP,5664.000000,1.879542e+07
26340,2015,48485,SWPP,131034.000000,1.879542e+07
26341,2015,48487,SWPP,13067.000000,1.879542e+07
26342,2015,48499,SWPP,43119.000000,1.879542e+07


In [7]:
ba_combined_df

,Balancing Authority,UTC Time at End of Hour,Demand (MW)
0,AEC,01/01/2018 7:00:00 AM,748
1,AEC,01/01/2018 8:00:00 AM,763
2,AEC,01/01/2018 9:00:00 AM,789
3,AEC,01/01/2018 10:00:00 AM,814
4,AEC,01/01/2018 11:00:00 AM,848
...,...,...,...
3158641,YAD,01/01/2019 1:00:00 AM,NaN
3158642,YAD,01/01/2019 2:00:00 AM,NaN
3158643,YAD,01/01/2019 3:00:00 AM,NaN
3158644,YAD,01/01/2019 4:00:00 AM,NaN


In [24]:
import pandas as pd

# Assuming ba_combined_df, serives_area_df, and aggregated_df are already defined
# ba_combined_df needs 'UTC Time at End of Hour', 'Balancing Authority', and 'Demand (MW)' columns
# serives_area_df should already have the 'Total_Population' calculated for each BA_Code and Year combination

# Step 1: Create a Year column in ba_combined_df
ba_combined_df['Year'] = pd.to_datetime(ba_combined_df['UTC Time at End of Hour']).dt.year

# Prepare the result list
expanded_results = []
# Initialize a set to store details of records where a match is not found or total_population is zero
unmatched_records = set()

# Step 2 & 3: Iterate through ba_combined_df and expand with matching records
for index, row in ba_combined_df.iterrows():
    year = row['Year']
    ba_code = row['Balancing Authority']

    try:
        # Attempt to directly convert or replace commas and then convert
        demand_mw = float(str(row['Demand (MW)']).replace(',', ''))
    except ValueError:
        print(f"Cannot convert Demand (MW) to float for BA {ba_code} at {row['UTC Time at End of Hour']}. Using 0 as placeholder.")
        demand_mw = 0  # Use a placeholder value or handle this scenario as appropriate
    
    
    # Find matching rows in serives_area_df
    matching_rows = serives_area_df[(serives_area_df['BA_Code'] == ba_code) & (serives_area_df['Year'] == year)]

    if matching_rows.empty:
        # If no matching rows are found, add the details to the unmatched_records set
        unmatched_records.add((year, ba_code, 'No FIPS - no matching row', demand_mw))
        continue  # Skip to the next row in ba_combined_df
    population = float(match_row['Population']) if pd.notna(match_row['Population']) else 0
    total_population = float(match_row['Total_Population']) if pd.notna(match_row['Total_Population']) else 0
    
    for _, match_row in matching_rows.iterrows():
        population = match_row['Population']
        total_population = match_row['Total_Population']
        
        if total_population > 0:  # Avoid division by zero
            adjusted_demand = (population / total_population) * demand_mw
            expanded_results.append({
                'UTC Time at End of Hour': row['UTC Time at End of Hour'],
                'County_FIPS': match_row['County_FIPS'],
                'BA_Code': ba_code,
                'Demand': adjusted_demand
            })
        else:
            # Handle the case of zero total population
            unmatched_records.add((year, ba_code, match_row['County_FIPS'], demand_mw))
            print(f'Error with {row["UTC Time at End of Hour"]} for county {match_row["County_FIPS"]} - zero total population.')

# Optionally, after processing
for unmatched in unmatched_records:
    print(f"Unmatched record: Year {unmatched[0]}, BA_Code {unmatched[1]}, County_FIPS {unmatched[2]}, Demand_MW {unmatched[3]}")

# Step 4: Convert expanded_results to DataFrame
expanded_results_df = pd.DataFrame(expanded_results)

# Optional: You might want to group by 'UTC Time at End of Hour', 'County_FIPS', 'BA_Code' and sum 'Demand' if necessary
# This step depends on whether you want to aggregate demands at this level
final_results_df = expanded_results_df.groupby(['UTC Time at End of Hour', 'County_FIPS', 'BA_Code'], as_index=False)['Demand'].sum()

# Save to CSV
final_results_df.to_csv(data_and_output_dir + 'output/final_expanded_results.csv', index=False)
